# Find Course Slots

In [1]:
#!/usr/bin/env python
# coding: utf8
"""Example of training an additional entity type
This script shows how to add a new entity type to an existing pretrained NER
model. To keep the example short and simple, only four sentences are provided
as examples. In practice, you'll need many more — a few hundred would be a
good start. You will also likely need to mix in examples of other entity
types, which might be obtained by running the entity recognizer over unlabelled
sentences, and adding their annotations to the training set.
The actual training is performed by looping over the examples, and calling
`nlp.entity.update()`. The `update()` method steps through the words of the
input. At each word, it makes a prediction. It then consults the annotations
provided on the GoldParse instance, to see whether it was right. If it was
wrong, it adjusts its weights so that the correct action will score higher
next time.
After training your model, you can save it to a directory. We recommend
wrapping models as Python packages, for ease of deployment.
For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities
Compatible with: spaCy v2.1.0+
Last tested with: v2.1.0
"""
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# new entity label
CLASS_LABEL = "CLASS"
ROOM_LABEL = "ROOM"

# training data
# Note: If you're using an existing model, make sure to mix in examples of
# other entity types that spaCy correctly recognized before. Otherwise, your
# model might learn the new type, but "forget" what it previously knew.
# https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting
TRAIN_DATA = [
    (
        "What time is CS 151?",
        {"entities": [(13, 18, CLASS_LABEL)]},
    ),
    (
        "What time is CS 111 G?",
        {"entities": [(13, 20, CLASS_LABEL)]},
    ),
    (
        "Which days are CS Principles",
        {"entities": [(15,28,CLASS_LABEL)]}
    ),
    (
        "What days are Python Programming 1",
        {"entities": [(14,33,CLASS_LABEL)]}
    ),
    (
        "What days are R Programming",
        {"entities": [(14,27,CLASS_LABEL)]}
    ),
    (
        "What days are Intro to Computer Sci",
        {"entities": [(14,35,CLASS_LABEL)]}
    ),
    
    (
        "What days are Intro to Computer Science",
        {"entities": [(14,39,CLASS_LABEL)]}
    ),
    (
        "Compilers",
        {"entities": [(0, 9, CLASS_LABEL)]},
    ),
    
    (
        "I have databases today", 
        {"entities": [(7, 16, CLASS_LABEL)]}
    ),
    (
        "I have databases 1 today", 
        {"entities": [(7, 18, CLASS_LABEL)]}
    ),
    (
        "Is CS 272 offered next fall?",
        {"entities": [(3, 9, CLASS_LABEL)]},
    ),
    (
        "How many sections of algorithms are there?", 
        {"entities": [(21, 31, CLASS_LABEL)]}),
]




In [2]:
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(model=None, new_model_name="class", output_dir=None, n_iter=30):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label(CLASS_LABEL)  # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

    # test the trained model
    test_text = "Where is Introduction to Philosophy?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)
        
    test_text = "Is CS 448 offered next fall?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)
        
    test_text = "Is datamining a class here?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)




In [3]:
main()

Created blank 'en' model
Losses {'ner': 39.105169385671616}
Losses {'ner': 16.419208173352672}
Losses {'ner': 14.188175948758285}
Losses {'ner': 11.658798483622402}
Losses {'ner': 19.179126334297145}
Losses {'ner': 15.790668496886445}
Losses {'ner': 10.250244756236384}
Losses {'ner': 6.91212903277571}
Losses {'ner': 8.52929037457056}
Losses {'ner': 6.469705222217781}
Losses {'ner': 9.110736854183717}
Losses {'ner': 9.547540094972067}
Losses {'ner': 2.9411224454141127}
Losses {'ner': 13.95807674832088}
Losses {'ner': 3.944174093150746}
Losses {'ner': 2.445253801785156}
Losses {'ner': 1.878979024968361}
Losses {'ner': 0.9550409504157803}
Losses {'ner': 0.1114455646493312}
Losses {'ner': 0.0002745951497976543}
Losses {'ner': 0.06186119927974294}
Losses {'ner': 0.00036634495997681365}
Losses {'ner': 0.010795733728274451}
Losses {'ner': 1.4259442929676103}
Losses {'ner': 8.81676463187457e-05}
Losses {'ner': 0.00019839945450949483}
Losses {'ner': 1.2244237948788084}
Losses {'ner': 5.66655841